In [44]:
version = 10290  # eps = 0.001
# version = 9259  # eps = 0.1
version =74649  # k = 3
# version = 11566  # k = 4
# version = 21622 # M
# version = 72646 # k = 5
# version = 93703 # M, k=3
# version = 56780 # M, k=4

In [45]:
import torch
import numpy as np
import spock_reg_model

In [46]:
labels = ['time', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno', 'a1', 'e1', 'i1', 'cos_Omega1', 'sin_Omega1', 'cos_pomega1', 'sin_pomega1', 'cos_theta1', 'sin_theta1', 'a2', 'e2', 'i2', 'cos_Omega2', 'sin_Omega2', 'cos_pomega2', 'sin_pomega2', 'cos_theta2', 'sin_theta2', 'a3', 'e3', 'i3', 'cos_Omega3', 'sin_Omega3', 'cos_pomega3', 'sin_pomega3', 'cos_theta3', 'sin_theta3', 'm1', 'm2', 'm3', 'nan_mmr_near', 'nan_mmr_far', 'nan_megno']

# not all of these labels are actually used. for training, these inputs are zeroed out, but still passed in as zeroes.
# ideally, the linear layer ignores them, which does happen if i do l1 regularization to it
skipped = ['nan_mmr_near', 'nan_mmr_far', 'nan_megno', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno']

In [47]:
scale = np.array([2.88976974e+03, 6.10019661e-02, 4.03849732e-02, 4.81638693e+01,
                   6.72583662e-02, 4.17939679e-02, 8.15995339e+00, 2.26871589e+01,
                   4.73612029e-03, 7.09223721e-02, 3.06455099e-02, 7.10726478e-01,
                   7.03392022e-01, 7.07873597e-01, 7.06030923e-01, 7.04728204e-01,
                   7.09420909e-01, 1.90740659e-01, 4.75502285e-02, 2.77188320e-02,
                   7.08891412e-01, 7.05214134e-01, 7.09786887e-01, 7.04371833e-01,
                   7.04371110e-01, 7.09828420e-01, 3.33589977e-01, 5.20857790e-02,
                   2.84763136e-02, 7.02210626e-01, 7.11815232e-01, 7.10512240e-01,
                   7.03646004e-01, 7.08017286e-01, 7.06162814e-01, 2.12569430e-05,
                   2.35019125e-05, 2.04211110e-05, 7.51048890e-02, 3.94254400e-01,
                   7.11351099e-02])

mean = np.array([ 4.95458585e+03,  5.67411891e-02,  3.83176945e-02,  2.97223474e+00,
                   6.29733979e-02,  3.50074471e-02,  6.72845676e-01,  9.92794768e+00,
                   9.99628430e-01,  5.39591547e-02,  2.92795061e-02,  2.12480714e-03,
                  -1.01500319e-02,  1.82667162e-02,  1.00813201e-02,  5.74404197e-03,
                   6.86570242e-03,  1.25316320e+00,  4.76946516e-02,  2.71326280e-02,
                   7.02054326e-03,  9.83378673e-03, -5.70616748e-03,  5.50782881e-03,
                  -8.44213953e-04,  2.05958338e-03,  1.57866569e+00,  4.31476211e-02,
                   2.73316392e-02,  1.05505555e-02,  1.03922250e-02,  7.36865006e-03,
                  -6.00523246e-04,  6.53016990e-03, -1.72038113e-03,  1.24807860e-05,
                   1.60314173e-05,  1.21732696e-05,  5.67292645e-03,  1.92488263e-01,
                   5.08607199e-03])

In [48]:
# for each label, pretty print the mean and scale
for l, m, s in zip(labels, mean, scale):
    print(f"{l}: mean={m:.2e}, scale={s:.2e}")

time: mean=4.95e+03, scale=2.89e+03
e+_near: mean=5.67e-02, scale=6.10e-02
e-_near: mean=3.83e-02, scale=4.04e-02
max_strength_mmr_near: mean=2.97e+00, scale=4.82e+01
e+_far: mean=6.30e-02, scale=6.73e-02
e-_far: mean=3.50e-02, scale=4.18e-02
max_strength_mmr_far: mean=6.73e-01, scale=8.16e+00
megno: mean=9.93e+00, scale=2.27e+01
a1: mean=1.00e+00, scale=4.74e-03
e1: mean=5.40e-02, scale=7.09e-02
i1: mean=2.93e-02, scale=3.06e-02
cos_Omega1: mean=2.12e-03, scale=7.11e-01
sin_Omega1: mean=-1.02e-02, scale=7.03e-01
cos_pomega1: mean=1.83e-02, scale=7.08e-01
sin_pomega1: mean=1.01e-02, scale=7.06e-01
cos_theta1: mean=5.74e-03, scale=7.05e-01
sin_theta1: mean=6.87e-03, scale=7.09e-01
a2: mean=1.25e+00, scale=1.91e-01
e2: mean=4.77e-02, scale=4.76e-02
i2: mean=2.71e-02, scale=2.77e-02
cos_Omega2: mean=7.02e-03, scale=7.09e-01
sin_Omega2: mean=9.83e-03, scale=7.05e-01
cos_pomega2: mean=-5.71e-03, scale=7.10e-01
sin_pomega2: mean=5.51e-03, scale=7.04e-01
cos_theta2: mean=-8.44e-04, scale=7.04

In [49]:
# load when on the cluster
model = spock_reg_model.load(version)
feature_nn = model.feature_nn

In [50]:
if model.hparams['combined_mass_feature']:
    print('adding combined mass label')
    labels.append('(m1+m2+m3)')

In [51]:
# load when local
# feature_nn = torch.load(f'models/{version}_feature_nn.pt')

In [52]:
input_linear = feature_nn.linear.weight * feature_nn.mask
input_linear = input_linear.detach().numpy()
if feature_nn.linear.bias is not None:
    input_bias = feature_nn.linear.bias.detach().numpy()
else:
    input_bias = np.zeros(input_linear.shape[0])

In [53]:
# m_i is the mean of the i'th feature, s_i is the standard deviation
# get the linear transformation that creates feature i
def linear_transformation(i):
    return input_linear[i]

In [54]:
# let's make the linear transformation a bit easier to read
def format_num(x):
    if abs(x) > 0.1:
        return f'{x:.2f}'
    if abs(x) > 0.01:
        return f'{x:.3f}'
    elif abs(x) > 0.001:
        return f'{x:.4f}'
    else:
        return f'{x:.2e}'

format_vec = np.vectorize(format_num)

In [55]:
# now we can write it as a combination of the input features
# we'll sort the features by their absolute value to make it a bit easier to read
def feature_equation(i):
    transformation = linear_transformation(i)
    bias = input_bias[i]
    sorted_ixs = np.argsort(np.abs(transformation))[::-1]
    features = [format_num(transformation[i]) + ' * ' + labels[i] for i in sorted_ixs if transformation[i] != 0]
    if bias != 0:
        features = [format_num(bias)] + features

    return features

In [56]:
for i in range(input_linear.shape[0]):
    print(str(i) + ": " + ' + '.join(feature_equation(i)))

0: 1.11 * m3 + -0.69 * m1 + 0.023 * e3
1: -1.19 * e2 + -0.89 * e1 + 0.49 * e3
2: 1.27 * m1 + -1.05 * a3 + 0.88 * m3
3: -0.84 * sin_Omega2 + 0.37 * sin_Omega3 + -0.0096 * cos_Omega1
4: 4.81 * a3 + -4.47 * a2 + -0.0049 * m3
5: 2.21 * a2 + -0.98 * a1 + 0.0084 * sin_theta2
6: -1.01 * i3 + 0.79 * i1 + 0.0054 * cos_pomega1
7: 0.90 * e2 + -0.37 * i1 + -0.31 * i2
8: -1.42 * m2 + 0.21 * e3 + 0.059 * e1
9: -1.20 * e3 + 0.77 * m1 + 0.33 * m3


In [57]:
def convert_to_latex(label):
    # 1. if it ends in a number, add an underscore
    if label[-1].isdigit():
        label = label[:-1] + '_' + label[-1]
    # 2. replace sin/cos with \sin/\cos
    label = label.replace('sin', '\\sin')
    label = label.replace('cos', '\\cos')
    label = label.replace('_Omega', '\\Omega')
    label = label.replace('_pomega', '\\omega')
    label = label.replace('_theta', '\\theta')
    return label

latex_labels = [convert_to_latex(label) for label in labels]
print(latex_labels)

['time', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno', 'a_1', 'e_1', 'i_1', '\\cos\\Omega_1', '\\sin\\Omega_1', '\\cos\\omega_1', '\\sin\\omega_1', '\\cos\\theta_1', '\\sin\\theta_1', 'a_2', 'e_2', 'i_2', '\\cos\\Omega_2', '\\sin\\Omega_2', '\\cos\\omega_2', '\\sin\\omega_2', '\\cos\\theta_2', '\\sin\\theta_2', 'a_3', 'e_3', 'i_3', '\\cos\\Omega_3', '\\sin\\Omega_3', '\\cos\\omega_3', '\\sin\\omega_3', '\\cos\\theta_3', '\\sin\\theta_3', 'm_1', 'm_2', 'm_3', 'nan_mmr_near', 'nan_mmr_far', 'nan_megno']


In [58]:
def latex_line(i):
    transformation = linear_transformation(i)
    bias = input_bias[i]
    sorted_ixs = np.argsort(np.abs(transformation))[::-1]
    features = [format_num(transformation[i]) + ' ' + latex_labels[i] for i in sorted_ixs if transformation[i] != 0]
    if bias != 0:
        features = [format_num(bias)] + features

    line = ' + '.join(features)
    line = line.replace('+ -', '- ')
    return line

def latex_string():
    s = ('\\begin{align*}\n'
        + f've&rsion={version}\\\\'
        + 'f_1& \\text{ features:} \\\\ \n')

    for i in range(input_linear.shape[0]):
        s += f'    &{i}: {latex_line(i)} \\\\ \n'

    s += '''\end{align*}'''
    return s


In [59]:
from IPython.display import display, Markdown
display(Markdown('$$\n' + latex_string() + '\n$$'))

$$
\begin{align*}
ve&rsion=74649\\f_1& \text{ features:} \\ 
    &0: 1.11 m_3 - 0.69 m_1 + 0.023 e_3 \\ 
    &1: -1.19 e_2 - 0.89 e_1 + 0.49 e_3 \\ 
    &2: 1.27 m_1 - 1.05 a_3 + 0.88 m_3 \\ 
    &3: -0.84 \sin\Omega_2 + 0.37 \sin\Omega_3 - 0.0096 \cos\Omega_1 \\ 
    &4: 4.81 a_3 - 4.47 a_2 - 0.0049 m_3 \\ 
    &5: 2.21 a_2 - 0.98 a_1 + 0.0084 \sin\theta_2 \\ 
    &6: -1.01 i_3 + 0.79 i_1 + 0.0054 \cos\omega_1 \\ 
    &7: 0.90 e_2 - 0.37 i_1 - 0.31 i_2 \\ 
    &8: -1.42 m_2 + 0.21 e_3 + 0.059 e_1 \\ 
    &9: -1.20 e_3 + 0.77 m_1 + 0.33 m_3 \\ 
\end{align*}
$$

In [35]:
version = 24880
model = spock_reg_model.load(version)
feature_nn = model.feature_nn

input_linear = feature_nn.linear.weight * feature_nn.mask
input_linear = input_linear.detach().numpy()
if feature_nn.linear.bias is not None:
    input_bias = feature_nn.linear.bias.detach().numpy()
else:
    input_bias = np.zeros(input_linear.shape[0])

In [36]:
def get_nonzero(arr):
    return arr[arr.nonzero()], arr.nonzero()

def set_nonzero(arr, values, indices):
    arr[indices] = values

In [37]:
def simplify(x, y, epsilon=0.1, zeroing_allowed=True):
    if x == 0 and y == 0:
        return (0, 0, 1), 0
    if x == 0:
        return (0, 1, 1/y), 0
    if y == 0:
        return (1, 0, 1/x), 0

    best_simplification, best_magnitude, best_error = None, None, None
    possible_values = list(range(-10, 11))
    candidate_ratios = [(a, c) for a in possible_values for c in possible_values]

    for a, b in candidate_ratios:
        if not zeroing_allowed and (a == 0 or b == 0):
            continue

        k = 1
        if a != 0:
            k = x / a
        if b != 0 and (a == 0 or abs(y) > abs(x)):
            k = y / b

        if k < 0: continue
        x2, y2 = k * a, k * b

        error = abs(x - x2) + abs(y - y2)
        # should be measured with the normalized values, not the original.
        if error >= epsilon: continue

        magnitude = abs(a) + abs(b)

        if best_error is None or magnitude < best_magnitude or magnitude == best_magnitude and error < best_error:
            best_simplification, best_error, best_magnitude = (a, b, x2, y2), error, magnitude

    return best_simplification, best_error

In [38]:
def simplify_stuff(input_linear, epsilon=0.1, normalize=True, zeroing_allowed=True):
    input_linear2 = input_linear.copy()
    for i in range(input_linear.shape[0]):
        nonzero, indices = get_nonzero(input_linear[i])
        x, y = nonzero
        print('original:\t', f'{x:.3f} {y:.3f}')

        if normalize:
            l1 = abs(x) + abs(y)
            x, y = x / l1, y / l1
            print('normalized:\t', f'{x:.3f} {y:.3f}')

        simplification, error = simplify(x, y, epsilon=epsilon, zeroing_allowed=zeroing_allowed)
        if simplification is None:
            print("no simplification found")
            nonzero = [x, y]
        else:
            a, b, x2, y2 = simplification
            print("new values:\t", f"{x2:.3f} {y2:.3f}", "with error", f"{error:.3f}")
            print("ratio:\t\t", f"{a} {b}")
            nonzero = [x2, y2]

            if normalize:
                l1 = abs(x2) + abs(y2)
                x2, y2 = x2 / l1, y2 / l1
                print("final normed:\t", f"{x2:.3f} {y2:.3f}")
                nonzero = [x2, y2]


        print()
        set_nonzero(input_linear2[i], nonzero, indices)

    input_linear2 = torch.tensor(input_linear2)
    feature_nn.linear.weight = torch.nn.Parameter(input_linear2)

    s = '24880_feature_nn_simplified_v3_'
    # if normalize:
        # s += 'norm_'
    # if not zeroing_allowed:
        # s += 'nozero_'
    s += f'eps={epsilon}.pt'

    torch.save(feature_nn, s)
    print(f'saved to', s)

In [39]:
# simplify_stuff(input_linear, epsilon=0.001, normalize=True, zeroing_allowed=False)